In [4]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.mask import mask
import numpy as np
from rasterstats import zonal_stats
from shapely.geometry import box

In [2]:
# Path to your raster and vector data
raster_path = r'F:\wenqu\Aviris\seven_sites_trait_map\site3b_d13c_std.tif'
vector_path = r'F:\wenqu\environment_factors\veg\veg_shapefiles.shp'

In [5]:
# 读取栅格数据
with rasterio.open(raster_path) as src:
    raster = src.read(1)  # 读取第一个波段
    raster_bounds = src.bounds  # 获取栅格数据的边界

    # 创建一个代表栅格边界的多边形
    raster_boundary = box(*raster_bounds)

    # 读取矢量数据
    gdf = gpd.read_file(vector_path)

    # 确保栅格和矢量数据在同一个坐标参考系统
    if gdf.crs != src.crs:
        gdf = gdf.to_crs(src.crs)

    # 筛选与栅格有交集的多边形
    gdf = gdf[gdf.intersects(raster_boundary)]

    # 为筛选后的多边形计算区域统计
    stats = zonal_stats(gdf, raster, affine=src.transform, stats="mean")

# 将均值添加到 GeoDataFrame
gdf['mean'] = [stat['mean'] for stat in stats]

# 查看结果
print(gdf)

C:\Users\15086\anaconda3\envs\wenqu_gdal\lib\site-packages\rasterstats\io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


    FID_NOAK_S              ECOREGION        PHYSIOGRAP  \
4         1255  Southern Brooks Range        Floodplain   
8          103  Northern Brooks Range  Glaciated Valley   
27        1374  Northern Brooks Range        Floodplain   
28        1375  Northern Brooks Range  Mountain, Rugged   
29        1376  Northern Brooks Range  Glaciated Valley   
30        1384  Northern Brooks Range  Mountain, Rugged   
32        2229  Southern Brooks Range  Glaciated Valley   
35        1027  Southern Brooks Range  Mountain, Rugged   
38        1098  Southern Brooks Range  Mountain, Rugged   
43        1009  Southern Brooks Range  Mountain, Rugged   

                    LITHOLOGY                             SUBSECTION  \
4          Silt, sand, gravel           Koyukuk Mountain Floodplains   
8                   Diamicton  Sagavanirktok Young Glaciated Valleys   
27         Silt, sand, gravel         Chandalar Mountain Floodplains   
28  Sedimentary, noncarbonate                Koyukuk Crest Mou

In [6]:
# Specify the path for your output CSV file
output_csv_path = r'F:\wenqu\Aviris\zonal_statistics\site3b_d13c_std.csv'

# Export the Geopandas DataFrame to a CSV
gdf.to_csv(output_csv_path, index=False)


In [2]:
shapefile = gpd.read_file(r'F:\wenqu\environment_factors\veg\veg_shapefiles.shp')
print("Original CRS of shapefile:", shapefile.crs)

Original CRS of shapefile: EPSG:32605


In [3]:
# Open the raster file
with rasterio.open(r'F:\wenqu\Aviris\seven_sites_trait_map\site3b_d13c_std.tif') as raster:
    raster_crs = raster.crs

In [4]:
raster_crs

CRS.from_epsg(32605)

In [3]:
zones = zones.to_crs(raster.crs)

In [4]:
print(raster.bounds)
print(zones.total_bounds)


BoundingBox(left=629869.750749821, bottom=7534979.225544035, right=645434.0635935275, top=7563798.51777)
[ 533390.42305868 7396099.04647997  740727.21380095 7827544.18449133]


In [5]:

# Open the raster dataset
with rasterio.open(r'F:\wenqu\Aviris\seven_sites_trait_map\site3b_d13c_std.tif') as dataset:
    # Get the no-data value
    no_data_value = dataset.nodata

print(f"The no-data value is: {no_data_value}")


The no-data value is: None


In [3]:
transform = raster.transform
rasterized_zones = rasterize(
    [(geom, value) for geom, value in zip(zones.geometry, zones['FID_NOAK_S'])],
    out_shape=raster.shape,
    transform=transform
)


In [4]:
stats = {}
for fid in zones['FID_NOAK_S'].unique():
    # Mask the raster data with each zone
    masked_data, _ = mask(raster, zones[zones['FID_NOAK_S'] == fid].geometry, crop=True)
    stats[value] = masked_data.mean()



ValueError: Input shapes do not overlap raster.